# Trabajo Práctico N° 2

## Información

Alumno: Ignacio Eloy González.

Profesores: Juan Pablo Manson, Alan Geary, Constantino Ferrucci, Dolores Sollberger

Año: 2024

## Librerias

In [ ]:
%%capture
!pip install chromadb spacy pandas rdflib gdown langchain-text-splitters PyPDF2 scikit-learn sentence-transformers langchain_experimental langchain-huggingface selenium langdetect
!pip install requests beautifulsoup4 torch transformers deep_translator pytextrank FlagEmbedding unidecode llama-index-llms-ollama wikipedia litellm[proxy] llama-index rank-bm25 nltk

In [ ]:
!wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get -f install -y

Selecting previously unselected package google-chrome-stable.
(Reading database ... 123635 files and directories currently installed.)
Preparing to unpack google-chrome-stable_current_amd64.deb ...
Unpacking google-chrome-stable (131.0.6778.139-1) ...
dpkg: dependency problems prevent configuration of google-chrome-stable:
 google-chrome-stable depends on libvulkan1; however:
  Package libvulkan1 is not installed.

dpkg: error processing package google-chrome-stable (--install):
 dependency problems - leaving unconfigured
Processing triggers for man-db (2.10.2-1) ...
Errors were encountered while processing:
 google-chrome-stable
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Correcting dependencies... Done
The following additional packages will be installed:
  libvulkan1 mesa-vulkan-drivers
The following NEW packages will be installed:
  libvulkan1 mesa-vulkan-drivers
0 upgraded, 2 newly installed, 0 to remove and 49 not upgraded.
1 no

In [ ]:
!wget -q https://storage.googleapis.com/chrome-for-testing-public/131.0.6778.87/linux64/chromedriver-linux64.zip
!unzip -q chromedriver-linux64.zip -d /usr/bin/

In [ ]:
# Web scrapping
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup

# Bases de datos
import rdflib
import pandas as pd
from rdflib import Graph, Literal, Namespace, URIRef
import urllib.parse
from urllib.parse import unquote
import chromadb
from chromadb.config import Settings
import json
from chromadb import Documents, EmbeddingFunction, Embeddings

# Extracción y modificación de texto
from PyPDF2 import PdfReader
from langdetect import detect
from deep_translator import GoogleTranslator
from unidecode import unidecode
import nltk
from nltk.tokenize import word_tokenize
import re
nltk.download('punkt')
nltk.download('punkt_tab')
import spacy


# Propósitos generales, descarga de archivos, etc.
import time
from google.colab import files
import os
from typing import List, Dict, Any
import gdown
from google.colab import userdata

# Regresión logística y demás de ML
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# RAG y LLM
from FlagEmbedding import FlagReranker
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
from langchain_experimental.text_splitter import SemanticChunker
from langchain_huggingface import HuggingFaceEmbeddings


# Agente
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
from llama_index.core.agent.react.formatter import ReActChatFormatter
import wikipedia
import logging

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


## Carga de modelos

In [ ]:
modelo_embeddings = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
model_embedding_name = "sentence-transformers/all-MiniLM-L12-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_embedding_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [ ]:
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True)

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

# Ejercicio 1 - RAG

## Extracción de texto de las fuentes de información (NO ES NECESARIO EJECUTAR)

In [ ]:
url = 'https://misutmeeple.com/2018/01/resena-rajas-of-the-ganges/'
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

article = soup.find("article", class_="entry content-bg single-entry post-48501 post type-post status-publish format-standard has-post-thumbnail hentry category-resenas tag-colocacion-de-losetas tag-colocacion-de-trabajadores tag-dennis-lohausen tag-eurogames tag-inka-brand tag-juegos-de-tablero tag-markus-brand tag-resenas tag-tiradas-de-dados")
div_contenidos = article.find("div", class_="entry-content-wrap")
h2_tags = div_contenidos.find_all("h2")

rows = []

for h2 in h2_tags:
    h2_title = h2.text.strip()  # Título h2
    paragraphs = []  # Lista para juntar las descripciones
    h3_descriptions = {}  # Diccionario para los subtítulos h3 y sus descripciones
    h3_title = None  # Inicializar h3_title antes del ciclo

    # Buscar los elementos <p>, <h3>, <ol>, <ul> y <h2> siguientes a este h2
    for sibling in h2.find_next_siblings():
        if sibling.name == "p":  # Si es un párrafo, lo agregamos a la lista
            paragraphs.append(sibling.text.strip())
        elif sibling.name == "h3":  # Si encontramos un h3, almacenamos las descripciones y las vaciamos
            if paragraphs:
                # Juntar todas las descripciones con salto de línea entre ellas
                joined_paragraphs = "\n".join(paragraphs)  # Usamos "\n" para separar las descripciones
                h3_descriptions[h3_title] = joined_paragraphs  # Guardar como valor para el subtítulo h3
            h3_title = sibling.text.strip()  # Título del h3
            paragraphs = []  # Reiniciar la lista de párrafos para el siguiente subtítulo
        elif sibling.name in ["ol", "ul"]:  # Si encontramos un <ol> o <ul>, capturamos los <li>
            list_items = []
            counter = 1  # Inicializamos el contador para los <li>
            for li in sibling.find_all("li"):
                list_items.append(f"# {li.text.strip()}")  # Usamos "#" para cada <li>
                counter += 1  # Incrementar el contador
            # Unir los items de la lista con saltos de línea
            list_items_text = "\n".join(list_items)
            paragraphs.append(f"Lista:\n{list_items_text}")  # Agregar la lista a los párrafos
        elif sibling.name == "h2":  # Si encontramos otro h2, dejamos de capturar párrafos
            break

    # Si quedan párrafos después del último h3, agregarlos al diccionario
    if paragraphs and h3_title:  # Asegurarse de que h3_title esté definido
        joined_paragraphs = "\n".join(paragraphs)  # Juntar los párrafos restantes con salto de línea
        h3_descriptions[h3_title] = joined_paragraphs  # Guardar el último conjunto de párrafos bajo h3

    # Agregar los datos al diccionario de filas
    for h3, description in h3_descriptions.items():
        rows.append({"Título": h2_title, "Subtítulo": h3, "Descripción": description})

# Convertir a DataFrame
df_contenido = pd.DataFrame(rows)

url = 'https://misutmeeple.com/2018/01/resena-rajas-of-the-ganges/'

response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

article = soup.find("article", class_="entry content-bg single-entry post-48501 post type-post status-publish format-standard has-post-thumbnail hentry category-resenas tag-colocacion-de-losetas tag-colocacion-de-trabajadores tag-dennis-lohausen tag-eurogames tag-inka-brand tag-juegos-de-tablero tag-markus-brand tag-resenas tag-tiradas-de-dados")
div_contenidos = article.find("div", class_="entry-content-wrap")
h2_opinion = div_contenidos.find("h2", string="Opinión Personal")

h2_opinion = soup.find("h2", string="Opinión Personal")

opinion_content = ""

if h2_opinion:
    for sibling in h2_opinion.find_next_siblings():
        if sibling.name == "p":
            opinion_content += sibling.text.strip() + "\n\n"
        elif sibling.name == "h2":
            break

df_opinion = pd.DataFrame([{"Título": "Opinión Personal","Subtítulo": None, "Descripción": opinion_content.strip()}])

df_concatenado = pd.concat([df_contenido, df_opinion], ignore_index=True)

with open("Reseña_español.txt", "w", encoding="utf-8") as file:
    for index, row in df_concatenado.iterrows():
        # Escribir el título
        file.write(f"Título: {row['Título']}\n")

        # Escribir el subtítulo (puede ser None, en cuyo caso no escribir nada)
        if row['Subtítulo'] is not None:
            file.write(f"Subtítulo: {row['Subtítulo']}\n")

        # Escribir la descripción
        file.write(f"Descripción: {row['Descripción']}\n")

        # Añadir una línea en blanco entre cada entrada
        file.write("\n")


files.download("Reseña_español.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
url = 'https://boardgamereview.co.uk/game-reviews/rajas-of-the-ganges-board-game-review/'
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

article = soup.find("div", class_='module_column tb-column col-full tb_617z590 first')

divs_text = article.find_all("div", class_='tb_text_wrap')
par_content = ""


for div in divs_text:
  if div.find("h2"):
    h2_title = div.find("h2").text.strip()
    paragraphs = []  # Lista para juntar las descripciones
    texts = div.find_all("p")
    for text in texts:
      if text.text.strip() not in paragraphs:
        paragraphs.append(text.text.strip())
    par_content += h2_title + '\n' + "".join(paragraphs) + "\n"


article_conclusion = soup.find("div", class_='module_row themify_builder_row tb_has_section tb_section-finalthoughts tb_cx9g050 tf_w')

div_text = article_conclusion.find("div", class_='tb_text_wrap')

paragraphs = []  # Lista para juntar las descripciones
conclusion_h2 = div_text.find("h2").text.strip()
texts = div_text.find_all("p")
for text in texts:
  if text.text.strip() not in paragraphs:
    paragraphs.append(text.text.strip())

par_content += conclusion_h2 + '\n' + "".join(paragraphs) + "\n"

with open('reseña_ingles.txt', "w", encoding="utf-8") as file:
  file.write(par_content)

files.download('reseña_ingles.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
with open('Reseña_español.txt', 'r', encoding='utf-8') as file:
    text_esp = file.read()


In [ ]:
text_esp_split = text_esp.split('\n')

In [ ]:
texto_traducido = []
for parte_text in text_esp_split:
  parte_traducida = GoogleTranslator(source='es', target='en').translate(parte_text)
  texto_traducido.append(parte_traducida)

texto_traducido_txt = "\n".join(texto_traducido)

In [ ]:
with open('reseña_traducida.txt', "w", encoding="utf-8") as file:
  file.write(texto_traducido_txt)

files.download('reseña_traducida.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Descarga de archivos (NO ES NECESARIO EJECUTAR)

In [ ]:
url = 'https://drive.google.com/drive/folders/1Dx2emCerOoSx4Jjdg-hR1Lfnp5sDCivI?usp=sharing'

gdown.download_folder(url, quiet=True, output='info_bdd_vectorial')

['info_bdd_vectorial/helpROTG.pdf',
 'info_bdd_vectorial/huc_16_7370_Rajas_of_the_Ganges_Auflage_A_2017_Anleitung_EN_sc.pdf',
 'info_bdd_vectorial/Rajas_of_the_Ganges_-_Solo_AI.pdf',
 'info_bdd_vectorial/Rajas_of_the_Ganges_-_Solo_version_v1.1.pdf',
 'info_bdd_vectorial/Rajas_of_the_Ganges_Plain_and_Simple.pdf',
 'info_bdd_vectorial/Rajas_of_the_Ganges_Quick_Rules_Guide.pdf',
 'info_bdd_vectorial/reseña_ingles.txt',
 'info_bdd_vectorial/reseña_traducida.txt',
 'info_bdd_vectorial/Reseñas varias rotg.pdf',
 'info_bdd_vectorial/Strategy guide rotg.pdf']

## Funciones de limpieza, extracción y split de los datos (NO ES NECESARIO EJECUTAR)

In [ ]:
def clean_text_with_newlines(text):
    # Elimina saltos de línea intermedios que no terminan un párrafo
    lines = text.splitlines()
    cleaned_text = ' '.join(line.strip() for line in lines if line.strip())
    return cleaned_text

In [ ]:
def remove_extra_spaces(text):
    return ' '.join(text.split())

In [ ]:
def remove_cosas_raras(text):
  text = text.replace('\uf0b7', '')
  text = text.replace('\u200b', '')
  text = text.replace('\u200b\u200b', '')
  return text.replace('\uf0a7', '')

In [ ]:
def extraer_texto_pdf(pdf_path):
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PdfReader(pdf_file)
        text = ''
        for page in pdf_reader.pages:
            text += page.extract_text()
    text = clean_text_with_newlines(text)
    text = remove_extra_spaces(text)
    text = remove_cosas_raras(text)
    return text

In [ ]:
def extraer_texto_txt(txt_path):
  with open(txt_path, 'r') as file:
    text = file.read()
  text = clean_text_with_newlines(text)
  text = remove_extra_spaces(text)
  text = remove_cosas_raras(text)
  return text

In [ ]:
def crear_split(text):
    text_splitter = SemanticChunker(hf)
    chunks = text_splitter.split_text(text)
    return chunks

## Base de datos vectorial (NO ES NECESARIO EJECUTAR)

In [ ]:
class CustomEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        return [modelo_embeddings.encode(text).tolist() for text in input]

In [ ]:
client = chromadb.PersistentClient(path="./bdd_vectorial")


collection = client.get_or_create_collection("document_chunks", embedding_function=CustomEmbeddingFunction())

In [ ]:
documentos = []

archivos_pdfs = os.listdir('info_bdd_vectorial')
archivos_ord_pdfs = sorted(archivos_pdfs)

for archivo in archivos_ord_pdfs:
    if archivo.endswith('.pdf'):
        pdf_path = os.path.join('info_bdd_vectorial', archivo)
        text = extraer_texto_pdf(pdf_path)
        chunk = crear_split(text)
        documentos.append(chunk)
    if archivo.endswith('.txt'):
        txt_path = os.path.join('info_bdd_vectorial', archivo)
        text = extraer_texto_txt(txt_path)
        chunk = crear_split(text)
        documentos.append(chunk)




Rajas_of_the_Ganges_-_Solo_AI.pdf
Rajas_of_the_Ganges_-_Solo_version_v1.1.pdf
Rajas_of_the_Ganges_Plain_and_Simple.pdf
Rajas_of_the_Ganges_Quick_Rules_Guide.pdf
Reseñas varias rotg.pdf
Strategy guide rotg.pdf
helpROTG.pdf
huc_16_7370_Rajas_of_the_Ganges_Auflage_A_2017_Anleitung_EN_sc.pdf
reseña_ingles.txt
reseña_traducida.txt


In [ ]:
embedding_chunk = []
documento_embeddings = []

for documento in documentos:
  for chunk in documento:
    embedding_chunk.append((chunk, modelo_embeddings.encode(chunk)))

  documento_embeddings.append(embedding_chunk)
  embedding_chunk = []

In [ ]:
for doc_id, chunk_embedding in enumerate(documento_embeddings, start=1):
    chunks, embeddings = zip(*chunk_embedding)
    num_chunks = len(chunks)
    collection.add(
        documents=list(chunks),  # Los chunks del documento
        ids=[f"doc{doc_id}_chunk{i}" for i in range(num_chunks)]  # IDs únicos
    )

In [ ]:
vect_db = collection.get()

In [ ]:
with open('vect_db.json', 'w') as vect_db_json:
    json.dump(vect_db, vect_db_json)

In [ ]:
files.download('vect_db.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Base de datos de grafos (NO ES NECESARIO EJECUTAR)

In [ ]:
def encontrar_nombre_partner(text):
  text = text.replace(':', '')
  text = text.replace('-', '')
  text = text.replace('Related Microbadge', '')
  text = text.replace('fan', '')
  text = text.strip()
  return text

In [ ]:
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

In [ ]:
dicc_info_grafos = {}
entities = ['artist','designer','publisher','category','mechanic','family']
links_a_d = {}
dicc_info_grafos = {}

In [ ]:
driver = webdriver.Chrome(service=Service("/usr/bin/chromedriver-linux64/chromedriver"), options=options)

url_game = "https://boardgamegeek.com/boardgame/220877/rajas-of-the-ganges"
driver.get(url_game)
time.sleep(10)

credits_link = driver.find_element(By.XPATH, "//a[contains(@href, '/credits')]")
print(credits_link)
credits_link.click()

for entity in entities:
  lista_info = []
  lista_links = []
  elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '/boardgame{entity}/')]")
  if entity == 'artist' or entity == 'designer':
    for element in elements:
      entity_link = element.get_attribute("href")
      name = element.text
      if (name, entity_link) not in lista_links:
        lista_links.append((name, entity_link))
      else:
        continue
    links_a_d[entity] = lista_links
    dicc_info_grafos[entity] = []
  else:
    for element in elements:
      entity_text = element.text
      lista_info.append(entity_text)

    dicc_info_grafos[entity] = lista_info



for entity, name_links in links_a_d.items():
  partner = None
  name, links = zip(*name_links)
  for link in links:
    driver.get(link)
    time.sleep(5)
    indx = links.index(link)
    name_entity = name[indx]
    related_section = driver.find_elements(By.XPATH, "//strong[contains(text(), 'Related')]")
    if related_section:
      parent_p = related_section[0].find_element(By.XPATH, "./ancestor::p")
      partner = encontrar_nombre_partner(parent_p.text)

    awards_section = driver.find_elements(By.XPATH, "//a[contains(@href, '/boardgamehonor/')]")
    awards_data = [award.text for award in awards_section]


    link_linked_games = driver.find_element(By.XPATH, "//a[contains(@href, '/linkeditems')]").get_attribute("href")

    driver.get(link_linked_games)
    time.sleep(5)

    linked_games = driver.find_elements(By.XPATH, "//h3[@class='summary-item-title summary-item-title-separated fs-inherit mb-0']/a")[:10]
    linked_games_data = [game.text for game in linked_games]

    if partner and awards_data:
      dicc_info_grafos[entity].append({'name':name_entity, 'partner': partner, 'awards': awards_data, 'other games':linked_games_data})
    else:
      dicc_info_grafos[entity].append({'name':name_entity, 'partner': [], 'awards':[], 'other games': linked_games_data})


driver.get(url_game)
time.sleep(5)

more_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//div[@gk-expandable-geekitem='awards']//div[@class='opener']/a")))
more_button.click()

game_awards_section = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//a[contains(@href, '/boardgamehonor/')]")))
game_awards_data = [award.text for award in game_awards_section]


dicc_info_grafos['awards'] = game_awards_data


driver.quit()

<selenium.webdriver.remote.webelement.WebElement (session="7c5db89c077a4fb42ce122087f4bdfc6", element="f.F483ABD8286138ACD5D80E34D3C707D1.d.459C501B2696B7BF9AD400178D9F456C.e.72")>


In [ ]:
g = Graph()

rajas = Namespace("http://rajas_data.org")
g.bind("rajas", rajas)

def encode_to_uri(name):
    return urllib.parse.quote(name.replace(" ", "_"))

nodo_principal = encode_to_uri("Rajas of the Ganges")
g.add((rajas[nodo_principal], rdflib.RDF.type, rajas.Game))

counter = 0

for entity, item in dicc_info_grafos.items():
  if entity == 'artist' or entity == 'designer':
    for designer in item:
      designer_name = encode_to_uri(designer['name'])
      if entity == 'designer':
        g.add((rajas[designer_name], rdflib.RDF.type, rajas.Designer))
        g.add((rajas[nodo_principal], rajas.DesignedBy, rajas[designer_name]))


      if entity == 'artist':
        g.add((rajas[designer_name], rdflib.RDF.type, rajas.Artist))
        g.add((rajas[nodo_principal], rajas.ArtBy, rajas[designer_name]))

      if designer['partner']:
        partner_name = encode_to_uri(designer['partner'])
        g.add((rajas[designer_name], rajas.partner, rajas[partner_name]))

      if designer['awards']:
        for award in designer['awards']:
          designer_award = encode_to_uri(award)
          g.add((rajas[designer_name], rajas.Won, rajas[designer_award]))

      for game in designer['other games']:
        game_name = encode_to_uri(game)
        g.add((rajas[game_name], rdflib.RDF.type, rajas.OtherGame))
        g.add((rajas[designer_name], rajas.ParticipatedIn, rajas[game_name]))

  elif entity == 'category':
    for category in item:
      category_name = encode_to_uri(category)
      g.add((rajas[category_name], rdflib.RDF.type, rajas.Category))
      g.add((rajas[nodo_principal], rajas.HasCategory, rajas[category_name]))

  elif entity == 'mechanic':
    for mechanic in item:
      mechanic_name = encode_to_uri(mechanic)
      g.add((rajas[mechanic_name], rdflib.RDF.type, rajas.Mechanic))
      g.add((rajas[nodo_principal], rajas.HasMechanic, rajas[mechanic_name]))

  elif entity == 'family':
    for indx, family in enumerate(item):
      family_name_data = family.split(': ')[1]
      if indx == 1:
        family_name_data = encode_to_uri(family)
        g.add((rajas[family_name_data], rdflib.RDF.type, rajas.Family))
        g.add((rajas[nodo_principal], rajas.isFrom, rajas[family_name_data]))
      elif indx == 2 or indx == 3:
        family_name_data = encode_to_uri(family)
        g.add((rajas[family_name_data], rdflib.RDF.type, rajas.Family))
        g.add((rajas[nodo_principal], rajas.DigitalImplementation, rajas[family_name_data]))
      elif indx == 6:
        family_name_data = encode_to_uri(family)
        g.add((rajas[family_name_data], rdflib.RDF.type, rajas.Family))
        g.add((rajas[nodo_principal], rajas.Religion, rajas[family_name_data]))

  elif entity == 'publisher':
    for publisher in item:
      publisher_name = encode_to_uri(publisher)
      g.add((rajas[publisher_name], rdflib.RDF.type, rajas.Publisher))
      g.add((rajas[nodo_principal], rajas.PublishedBy, rajas[publisher_name]))

  elif entity == 'awards':
    for game_award in item:
      game_award_name = encode_to_uri(game_award)
      g.add((rajas[game_award_name], rdflib.RDF.type, rajas.Award))
      g.add((rajas[nodo_principal], rajas.Won, rajas[game_award_name]))


  counter += 1


base_de_datos_grafos = g.serialize(destination='bdd_grafos.rdf',format='turtle')

In [ ]:
files.download('bdd_grafos.rdf')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Base de datos tabular (NO ES NECESARIO EJECUTAR)

In [ ]:
dicc_bdd_tabular = {}


In [ ]:
driver = webdriver.Chrome(service=Service("/usr/bin/chromedriver-linux64/chromedriver"), options=options)

url_game = "https://boardgamegeek.com/boardgame/220877/rajas-of-the-ganges"
driver.get(url_game)
time.sleep(5)

game_stats = driver.find_element(By.XPATH, "//ul[@class='gameplay']")

diff_stats = game_stats.find_elements(By.XPATH, ".//li[@class='gameplay-item']")

for diff_stat in diff_stats:
  stat_title = diff_stat.find_element(By.XPATH, ".//h3[@class='sr-only']").text
  if stat_title == 'Number of Players' or stat_title == 'Play Time':
    min_stat = diff_stat.find_element(By.XPATH, ".//span[@ng-if='min > 0']").text
    max_stat = diff_stat.find_element(By.XPATH, ".//span[@ng-if='max>0 && min != max']").text
    dicc_bdd_tabular[stat_title] = min_stat + ' ' + max_stat
  elif stat_title == 'Suggested Age':
    stat = diff_stat.find_element(By.XPATH, ".//span[@itemprop='suggestedMinAge']").text
    dicc_bdd_tabular[stat_title] = stat
  else:
    continue

year_release = driver.find_element(By.XPATH, ".//span[@class='game-year ng-binding ng-scope']").text.replace('(', '').replace(')', '')

dicc_bdd_tabular['Release Year'] = year_release

stats_url = driver.find_element(By.XPATH, "//a[contains(@href, 'rajas-of-the-ganges/stats')]").get_attribute("href")

driver.get(stats_url)
time.sleep(5)

stats_bdd = ['Avg. Rating', 'Weight', 'Fans', 'Overall Rank', 'Strategy Rank', 'All Time Plays', 'This Month']


all_stats = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//li[contains(@class , 'outline-item')]")))

for stat in all_stats:
    stat_title = stat.find_element(By.XPATH, ".//div[contains(@class ,'outline-item-title')]").text
    try:
      stat_description = stat.find_element(By.XPATH, ".//div[contains(@class , 'outline-item-description')]/a").text
    except:
      stat_description = stat.find_element(By.XPATH, ".//div[contains(@class , 'outline-item-description')]").text

    if stat_title in stats_bdd:
      try:
        stat_description = stat_description.replace(',', '.')
      except:
        pass
      dicc_bdd_tabular[stat_title] = stat_description

driver.quit()

In [ ]:
bdd_tabular = pd.DataFrame(dicc_bdd_tabular, index=[0])

In [ ]:
bdd_tabular

,Number of Players,Play Time,Suggested Age,Release Year,Avg. Rating,Weight,Fans,Overall Rank,Strategy Rank,All Time Plays,This Month
0,2 –4,45 –75,12,2017,7.730,2.89 / 5,691,156,123,59.381,182


In [ ]:
bdd_tabular.to_csv('bdd_tabular.csv', index=False)

## Clasificador de prompts

### Clasificador con modelo propio

In [ ]:
data = [

    # Examples for the "graphs" class (label 0)
    {"query": "How do you play this game?", "label": 0},
    {"query": "What is the most effective strategy in this game?", "label": 0},
    {"query": "Explain the tiebreaker rules in this game.", "label": 0},
    {"query": "Where can I find a guide for advanced strategies?", "label": 0},
    {"query": "What does the official rulebook of the game include?", "label": 0},
    {"query": "How are the final rounds scored?", "label": 0},
    {"query": "What is the difference between the basic and advanced rules?", "label": 0},
    {"query": "What resources are essential to win?", "label": 0},
    {"query": "Explain how the turn order affects the game.", "label": 0},
    {"query": "Are there rule variants for fewer players?", "label": 0},
    {"query": "What strategies can I use to improve my performance in the early rounds?", "label": 0},
    {"query": "Explain the key differences between the versions of the rulebook.", "label": 0},
    {"query": "How is a tie resolved if it happens in the last round?", "label": 0},
    {"query": "What tips are there for new players in this game?", "label": 0},
    {"query": "Where can I find a detailed guide to the roles in the game?", "label": 0},
    {"query": "How are resources distributed at the beginning of the game?", "label": 0},
    {"query": "What is the best order to learn the advanced rules?", "label": 0},
    {"query": "What changes should I make if playing with less time than recommended?", "label": 0},
    {"query": "What variants exist for playing in solo mode?", "label": 0},
    {"query": "What does this game expansion include in terms of rules and mechanics?", "label": 0},
    {"query": "What are the rules for setting up the game?", "label": 0},
    {"query": "How do you calculate the final score in Rajas of the Ganges?", "label": 0},
    {"query": "What are the recommended strategies for beginners?", "label": 0},
    {"query": "What does the official rulebook say about tie-breaking?", "label": 0},
    {"query": "Where can I find a detailed guide on dice placement strategies?", "label": 0},
    {"query": "What are the advanced rules for resource management?", "label": 0},
    {"query": "How do the karma mechanics work in this game?", "label": 0},
    {"query": "What tips are there for using the marketplace tiles effectively?", "label": 0},
    {"query": "Are there any quick reference guides for the game?", "label": 0},
    {"query": "What strategies help in balancing fame and money tracks?", "label": 0},
    {"query": "What are some common mistakes players make?", "label": 0},
    {"query": "What is the best way to start building provinces?", "label": 0},
    {"query": "How do I efficiently manage workers during the mid-game?", "label": 0},
    {"query": "What are the key actions to prioritize in the early game?", "label": 0},
    {"query": "Is there a simplified version of the rules for children?", "label": 0},
    {"query": "What expansions add new mechanics to the game?", "label": 0},
    {"query": "How do the boats impact gameplay progression?", "label": 0},
    {"query": "Are there solo mode strategies available?", "label": 0},
    {"query": "What is the role of the palace tiles in advanced play?", "label": 0},
    {"query": "What reviews highlight the best strategic aspects of the game?", "label": 0},
    {"query": "How does scoring work in the advanced mode?", "label": 0},
    {"query": "What are the main objectives in this game?", "label": 0},
    {"query": "What are the key mechanics of this game?", "label": 0},
    {"query": "Where can I find an illustrated guide to the rules?", "label": 0},
    {"query": "What are the best strategies for resource management?", "label": 0},
    {"query": "How should I approach the first round in this game?", "label": 0},
    {"query": "What are the steps to set up the board?", "label": 0},
    {"query": "Are there any common beginner mistakes to avoid?", "label": 0},
    {"query": "What are the differences between solo and multiplayer modes?", "label": 0},
    {"query": "How do I use special abilities effectively?", "label": 0},
    {"query": "What should I prioritize when upgrading my resources?", "label": 0},
    {"query": "How do alliances work in cooperative play?", "label": 0},
    {"query": "Can you explain the scoring system in detail?", "label": 0},
    {"query": "What is the turn structure in this game?", "label": 0},
    {"query": "What are the most common winning strategies?", "label": 0},
    {"query": "Are there any expansions that change the core rules?", "label": 0},
    {"query": "What are the house rules people often use?", "label": 0},
    {"query": "How do I balance risk and reward in this game?", "label": 0},
    {"query": "What are some advanced tactics for experienced players?", "label": 0},
    {"query": "How does terrain affect gameplay?", "label": 0},
    {"query": "What are the best practices for managing resources?", "label": 0},
    {"query": "How do I handle a tie-breaker situation?", "label": 0},
    {"query": "Are there any official rule clarifications?", "label": 0},
    {"query": "What is the recommended order to teach the game?", "label": 0},
    {"query": "How can I adapt the game for younger players?", "label": 0},
    {"query": "What should I do if we misinterpret a rule mid-game?", "label": 0},
    {"query": "What scenarios work best for introducing new players?", "label": 0},
    {"query": "How do I keep the game balanced across different skill levels?", "label": 0},
    {"query": "What’s the recommended player count for the most fun?", "label": 0},
    {"query": "Are there any time-saving tips for setup?", "label": 0},
    {"query": "How do I resolve disputes over rule interpretations?", "label": 0},
    {"query": "What is the ideal way to plan moves ahead?", "label": 0},
    {"query": "Can you give an overview of the rulebook?", "label": 0},
    {"query": "How do bonuses work during scoring?", "label": 0},
    {"query": "What role does luck play in the overall strategy?", "label": 0},
    {"query": "How do I calculate resource efficiency?", "label": 0},
    {"query": "Are there any unwritten strategies that experienced players use?", "label": 0},
    {"query": "What are the key milestones to focus on in the mid-game?", "label": 0},
    {"query": "How do endgame conditions trigger?", "label": 0},
    {"query": "Are there optional rules that add complexity?", "label": 0},
    {"query": "What is the best way to handle disputes in cooperative mode?", "label": 0},
    {"query": "How do role cards affect gameplay?", "label": 0},
    {"query": "What is the best way to learn advanced mechanics?", "label": 0},
    {"query": "What are some effective ways to slow down opponents?", "label": 0},
    {"query": "What’s the easiest way to explain this game to new players?", "label": 0},
    {"query": "How can I make my moves more efficient?", "label": 0},
    {"query": "What are the common house rules for experienced groups?", "label": 0},
    {"query": "What kind of strategies work best in a long game?", "label": 0},
    {"query": "What are some situational strategies I should know?", "label": 0},
    {"query": "How do environmental factors like lighting affect gameplay?", "label": 0},
    {"query": "What are the specific scoring bonuses for achievements?", "label": 0},
    {"query": "What’s the best way to handle aggressive players?", "label": 0},
    {"query": "How do I speed up the game without losing balance?", "label": 0},
    {"query": "What’s the most effective way to maximize endgame points?", "label": 0},
    {"query": "What should I do if I fall behind early in the game?", "label": 0},
    {"query": "How do I manage high-risk, high-reward strategies?", "label": 0},
    {"query": "What are the key differences between competitive and cooperative modes?", "label": 0},
    {"query": "What should I consider when building my strategy for this game?", "label": 0},
    {"query": "How do I integrate expansions smoothly into gameplay?", "label": 0},
    {"query": "What are the pros and cons of each faction or role?", "label": 0},

    # Examples for the "graphs" class (label 1)
    {"query": "Who designed this game?", "label": 1},
    {"query": "Who did the artwork for this game?", "label": 1},
    {"query": "What awards has this designer won?", "label": 1},
    {"query": "What other games has this artist worked on?", "label": 1},
    {"query": "What other games have the designers worked on before?", "label": 1},
    {"query": "Who are the publishers of this game?", "label": 1},
    {"query": "What is the main category of this game?", "label": 1},
    {"query": "Where is this board game set?", "label": 1},
    {"query": "What is the relationship between the game designers?", "label": 1},
    {"query": "What religion influences the game?", "label": 1},
    {"query": "When was this game released?", "label": 1},
    {"query": "Are there connections between this game and others by the same author?", "label": 1},
    {"query": "What other games have been published by the same publisher?", "label": 1},
    {"query": "Who are the main collaborators in the game's design?", "label": 1},
    {"query": "What is the connection between this game and other similar-themed games?", "label": 1},
    {"query": "What awards has this artist received for their work?", "label": 1},
    {"query": "Is there a thematic relationship between this designer and others in the same genre?", "label": 1},
    {"query": "What is the main category of this game according to the publisher?", "label": 1},
    {"query": "What cultural elements influence the design of this game?", "label": 1},
    {"query": "What relationship have the designer and the artist?", "label": 1},
    {"query": "What designers have worked on games related to this one?", "label": 1},
    {"query": "What role does art play in the game's narrative?", "label": 1},
    {"query": "Who designed Rajas of the Ganges?", "label": 1},
    {"query": "What awards has the game won since its release?", "label": 1},
    {"query": "Who is the artist behind the game's visuals?", "label": 1},
    {"query": "What are the categories of the game?", "label": 1},
    {"query": "What is the cultural influence in the game?", "label": 1},
    {"query": "Who published the game, and where are they based?", "label": 1},
    {"query": "What is the relationship between the designers of the game?", "label": 1},
    {"query": "Are there connections between this game and others by the same team?", "label": 1},
    {"query": "What is the primary category for this board game?", "label": 1},
    {"query": "Which other games has the artist worked on?", "label": 1},
    {"query": "How is the theme of Hinduism reflected in the gameplay?", "label": 1},
    {"query": "What are the geographic settings depicted in the game?", "label": 1},
    {"query": "What other games have the designers worked on?", "label": 1},
    {"query": "Who collaborated on the expansion design?", "label": 1},
    {"query": "What awards have the designers received individually?", "label": 1},
    {"query": "How does the game's theme relate to Indian culture?", "label": 1},
    {"query": "What is the connection between the publisher and other games?", "label": 1},
    {"query": "Are there sequels or spin-offs related to Rajas of the Ganges?", "label": 1},
    {"query": "How do the designers’ styles differ from each other?", "label": 1},
    {"query": "What collaborations exist between this game's team and others?", "label": 1},
    {"query": "Who are the co-designers of this game?", "label": 1},
    {"query": "What games has the designer released recently?", "label": 1},
    {"query": "What is the connection between this designer and others in the same genre?", "label": 1},
    {"query": "Which publisher is responsible for distributing this game?", "label": 1},
    {"query": "What collaborations exist between this designer and artist?", "label": 1},
    {"query": "What cultural influences shaped the game's art?", "label": 1},
    {"query": "In which region is the game thematically set?", "label": 1},
    {"query": "What themes does the game's narrative explore?", "label": 1},
    {"query": "Which awards has this artist won for their work?", "label": 1},
    {"query": "What other games feature a similar artistic style?", "label": 1},
    {"query": "What relationship the designers have?", "label": 1},
    {"query": "Who are the primary contributors to the game's mechanics?", "label": 1},
    {"query": "What is the connection between this publisher and others in the industry?", "label": 1},
    {"query": "What specific techniques did the artist use in this game?", "label": 1},
    {"query": "Which games were released alongside this one by the same publisher?", "label": 1},
    {"query": "What is the religious context of this game's story?", "label": 1},
    {"query": "Which publisher distributes the game in different regions?", "label": 1},
    {"query": "What are the main inspirations behind the game's design?", "label": 1},
    {"query": "What other titles are categorized under the same theme?", "label": 1},
    {"query": "What role did the lead designer have in other projects?", "label": 1},
    {"query": "What are the unique traits of the artists involved?", "label": 1},
    {"query": "Who funded the development of this game?", "label": 1},
    {"query": "How does this game fit within the designer's portfolio?", "label": 1},
    {"query": "What are the notable features of the publisher's catalog?", "label": 1},
    {"query": "Which games share similar mechanics from this designer?", "label": 1},
    {"query": "Who were the artistic influences behind the game?", "label": 1},
    {"query": "What relationships exist between this artist and other illustrators?", "label": 1},
    {"query": "What are the defining characteristics of the category this game belongs to?", "label": 1},
    {"query": "Which category does the publisher officially classify the game in?", "label": 1},
    {"query": "What is the historical context represented in the game?", "label": 1},
    {"query": "What are the notable achievements of the game's developers?", "label": 1},
    {"query": "How many games has the publisher released in total?", "label": 1},
    {"query": "Which designer contributed to both this game and similar titles?", "label": 1},
    {"query": "What historical period does the game's theme represent?", "label": 1},
    {"query": "What cross-cultural elements are present in the game's narrative?", "label": 1},
    {"query": "How is the game categorized within its genre?", "label": 1},
    {"query": "What role does the publisher play in promoting this game?", "label": 1},
    {"query": "What are the connections between the lead designer and other studios?", "label": 1},
    {"query": "Which designers worked together on similar games?", "label": 1},
    {"query": "What is the artist's most acclaimed project?", "label": 1},
    {"query": "Who oversees the thematic consistency in this publisher's games?", "label": 1},
    {"query": "How many awards has this game's publisher won?", "label": 1},
    {"query": "What unique features distinguish this designer's games?", "label": 1},
    {"query": "What relationships link this publisher to other industry leaders?", "label": 1},
    {"query": "How does the game's art reflect its cultural themes?", "label": 1},
    {"query": "What timeline does the publisher follow for releases?", "label": 1},
    {"query": "What historical figures inspire this game's characters?", "label": 1},
    {"query": "Which other games have similar thematic elements?", "label": 1},
    {"query": "What innovations has the designer introduced to the genre?", "label": 1},
    {"query": "Which publisher editions are available globally?", "label": 1},
    {"query": "What genre-specific challenges does this game explore?", "label": 1},
    {"query": "What is the relationship between this artist and the game's theme?", "label": 1},
    {"query": "What unique marketing approaches has the publisher used?", "label": 1},
    {"query": "Who are the most prominent artists in this game's style?", "label": 1},
    {"query": "What are the regional influences on the game's development?", "label": 1},
    {"query": "What makes this game's category distinct from others?", "label": 1},
    {"query": "How is the artist's style reflected in the game's components?", "label": 1},
    {"query": "What partnerships exist between this publisher and others?", "label": 1},
    {"query": "Who are the supporting designers of this game?", "label": 1},

    # Examples for the "tabular" class (label 2)
    {"query": "What is the average rating of this game?", "label": 2},
    {"query": "How many monthly players does this game have?", "label": 2},
    {"query": "How difficult is it to learn this game?", "label": 2},
    {"query": "What is the global ranking of this game?", "label": 2},
    {"query": "How many fans does this game have in total?", "label": 2},
    {"query": "What is the ratio of fans to total players?", "label": 2},
    {"query": "How well-rated is this game among experts?", "label": 2},
    {"query": "How many average players participate per game?", "label": 2},
    {"query": "What is this game's position in the strategy ranking?", "label": 2},
    {"query": "Are there regional differences in the rating?", "label": 2},
    {"query": "How many players does the box recommend for optimal games?", "label": 2},
    {"query": "What is the average playtime according to statistics?", "label": 2},
    {"query": "How difficult is it to master the advanced mechanics?", "label": 2},
    {"query": "What is the number of active monthly players in this game?", "label": 2},
    {"query": "How does this game's rating compare to others in the same genre?", "label": 2},
    {"query": "What data is available about its popularity growth?", "label": 2},
    {"query": "What is the average rating among younger players?", "label": 2},
    {"query": "What statistical changes have occurred in the ranking this year?", "label": 2},
    {"query": "What is the average number of games played per week?", "label": 2},
    {"query": "How many official expansions exist for this game according to metrics?", "label": 2},
    {"query": "What is the average playtime according to user reviews?", "label": 2},
    {"query": "How many players does the box recommend for optimal gameplay?", "label": 2},
    {"query": "What is the difficulty rating of the game?", "label": 2},
    {"query": "What is the global ranking of Rajas of the Ganges?", "label": 2},
    {"query": "How many expansions exist for this game?", "label": 2},
    {"query": "What is the average rating from board game experts?", "label": 2},
    {"query": "How many awards has the game won in total?", "label": 2},
    {"query": "What is the average score required to win?", "label": 2},
    {"query": "What is the number of dice included in the base game?", "label": 2},
    {"query": "What percentage of reviews are highly positive?", "label": 2},
    {"query": "What is the median game length for 4 players?", "label": 2},
    {"query": "How does the game compare in rankings to other economic games?", "label": 2},
    {"query": "What is the estimated number of fans globally?", "label": 2},
    {"query": "How many tiles are included in the base game?", "label": 2},
    {"query": "What is the average cost of the game in different regions?", "label": 2},
    {"query": "How do player counts impact the average playtime?", "label": 2},
    {"query": "What percentage of users recommend the game?", "label": 2},
    {"query": "What are the statistical trends in the game’s popularity?", "label": 2},
    {"query": "How does the game perform in family-friendly rankings?", "label": 2},
    {"query": "What is the average number of players per session?", "label": 2},
    {"query": "What is the recommended age for players?", "label": 2},
    {"query": "How long is the average play session?", "label": 2},
    {"query": "What is the highest player rating this game has received?", "label": 2},
    {"query": "How many fans does this game have currently?", "label": 2},
    {"query": "What is the game's ranking among family board games?", "label": 2},
    {"query": "What is the average playtime per session?", "label": 2},
    {"query": "How many total players have logged games?", "label": 2},
    {"query": "What percentage of players recommend this game?", "label": 2},
    {"query": "How does the game's difficulty compare to similar titles?", "label": 2},
    {"query": "What is the most common rating given by players?", "label": 2},
    {"query": "How has the game's popularity changed over time?", "label": 2},
    {"query": "What is the median number of players per game?", "label": 2},
    {"query": "How many monthly active players does the game have?", "label": 2},
    {"query": "What is the maximum number of players supported?", "label": 2},
    {"query": "How does the average rating vary by region?", "label": 2},
    {"query": "What is the total number of expansions released?", "label": 2},
    {"query": "What is the game's ranking in cooperative board games?", "label": 2},
    {"query": "What percentage of games result in a tie?", "label": 2},
    {"query": "How often do players use house rules?", "label": 2},
    {"query": "What is the average win rate for experienced players?", "label": 2},
    {"query": "What is the most common game length reported?", "label": 2},
    {"query": "How does the game's rating differ between new and veteran players?", "label": 2},
    {"query": "What are the most common player demographics?", "label": 2},
    {"query": "What is the average game length across different group sizes?", "label": 2},
    {"query": "In what year was the game published?", "label": 2},
    {"query": "What is the average score per game?", "label": 2},
    {"query": "What is the standard deviation of player ratings?", "label": 2},
    {"query": "How many versions of this game exist globally?", "label": 2},
    {"query": "How has the player base evolved in the last five years?", "label": 2},
    {"query": "What is the game's peak number of concurrent players?", "label": 2},
    {"query": "How many hours does an average player spend on this game?", "label": 2},
    {"query": "What is the gender distribution among players?", "label": 2},
    {"query": "How frequently is this game played in competitions?", "label": 2},
    {"query": "What is the most frequently used strategy among top players?", "label": 2},
    {"query": "What is the minimum recommended number of players?", "label": 2},
    {"query": "How many copies of the game have been sold?", "label": 2},
    {"query": "What is the retention rate for new players?", "label": 2},
    {"query": "What is the average rating for the game’s expansions?", "label": 2},
    {"query": "What is the average time per turn?", "label": 2},
    {"query": "What is the typical number of rounds in a game?", "label": 2},
    {"query": "How many players rate this game as their favorite?", "label": 2},
    {"query": "What are the longest recorded playtimes?", "label": 2},
    {"query": "What is the correlation between player experience and scores?", "label": 2},
    {"query": "What is the average rating for strategy games in this category?", "label": 2},
    {"query": "What is the success rate for different strategies?", "label": 2},
    {"query": "How does the game rank in replayability?", "label": 2},
    {"query": "What is the average number of moves per game?", "label": 2},
    {"query": "How has the game performed in global rankings this year?", "label": 2},
    {"query": "What is the variance in average playtimes?", "label": 2},
    {"query": "How often do players report technical issues?", "label": 2},
    {"query": "What is the ratio of cooperative to competitive games?", "label": 2},
    {"query": "How does player satisfaction correlate with game length?", "label": 2},
    {"query": "What is the average number of expansions owned per player?", "label": 2},
    {"query": "What is the drop-off rate for first-time players?", "label": 2},
    {"query": "What is the impact of player count on average scores?", "label": 2},
    {"query": "How many awards has the game received overall?", "label": 2},
]


data_querys = pd.DataFrame(data=data)

In [ ]:
# Separamos las querys de las etiquetas
x = data_querys['query'].to_list()
y = data_querys['label'].to_list()

# Dividimos las querys para entrenamiento y testeo
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

# Generamos los embeddings de las querys para poder entrenar la regresión logística.
x_train_vect = modelo_embeddings.encode(x_train)
x_test_vect = modelo_embeddings.encode(x_test)


relog = LogisticRegression(max_iter=500, random_state=42)


relog.fit(x_train_vect, y_train)

y_pred_relog = relog.predict(x_test_vect)

In [ ]:
report_clf = classification_report(y_test, y_pred_relog, zero_division=1)

print("Reporte de clasificación Regresión Logística:\n", report_clf)

Reporte de clasificación Regresión Logística:
               precision    recall  f1-score   support

           0       0.96      1.00      0.98        24
           1       1.00      1.00      1.00        19
           2       1.00      0.94      0.97        17

    accuracy                           0.98        60
   macro avg       0.99      0.98      0.98        60
weighted avg       0.98      0.98      0.98        60



In [ ]:
query_clas = 'Where is the game from?'

query_clas_vect = modelo_embeddings.encode([query_clas])

y_pred_query = relog.predict(query_clas_vect)

print(y_pred_query)

[1]


### Clasificador con LLM (HUGGINGFACE_TOKEN O API_KEY)

In [ ]:
api_key_hf = userdata.get('HUGGINGFACE_TOKEN')

In [ ]:
from huggingface_hub import InferenceClient

llm_client = InferenceClient(api_key=api_key_hf)

def classify_query(query: str, model:str):

  messages = [
    {
      "role": "user",
      "content": f""" Classify this query with one of these classes: relations, statistics, reviews/rules. Return only one word, the class.
      What is the portuguese in the game? - reviews/rules
      How the karma affects the game? - reviews/rules
      Who did the artwork for this game? - relations
      what companies did published the game? - relations
      In what year was the game published? - statistics
      What is the recommended age for playing? - statistics
      {query} - """
    }
  ]

  completion = llm_client.chat.completions.create(
      model=model,
    messages=messages,
    max_tokens=500
  )

  return completion.choices[0].message.content

In [ ]:
query = 'what is the portuguese in the game?'

print(classify_query(query=query, model="Qwen/Qwen2.5-72B-Instruct"))


reviews/rules


In [ ]:
query = 'In what year was the game published?'

print(classify_query(query=query, model="Qwen/Qwen2.5-72B-Instruct"))


statistics


In [64]:
query = 'Who are the publishers of the game?'

print(classify_query(query=query, model="Qwen/Qwen2.5-72B-Instruct"))


relations


In [ ]:
query = 'in the scale of 1 to 5 what is the difficulty of the game?'

print(classify_query(query=query, model="Qwen/Qwen2.5-72B-Instruct"))

statistics


Comentamos las predicciones de test, ya que consume muchas llamadas a API de huggingface

In [ ]:
# predictions_test = []

# for query in x_test:
#   predictions_test.append(classify_query(query=query, model="Qwen/Qwen2.5-72B-Instruct"))

In [ ]:
# y_pred_llm = []
# for prediction, label in zip(predictions_test, y_test):
#   if prediction == 'relations':
#     y_pred_llm.append(1)

#   elif prediction == 'statistics':
#     y_pred_llm.append(2)
#   else:
#     y_pred_llm.append(0)




# report_llm = classification_report(y_test, y_pred_llm, zero_division=1)

# print("Reporte de clasificación de la LLM:\n", report_llm)

## Funciones de búsqueda de contexto

### Carga de bases de datos

In [ ]:
!wget https://raw.githubusercontent.com/nacho-gonz/NLP_TUIA_2024/refs/heads/main/vect_db.json

--2024-12-17 21:37:26--  https://raw.githubusercontent.com/nacho-gonz/NLP_TUIA_2024/refs/heads/main/vect_db.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 150192 (147K) [text/plain]
Saving to: ‘vect_db.json’

vect_db.json        100%[===================>] 146.67K  --.-KB/s    in 0.02s   

2024-12-17 21:37:27 (8.14 MB/s) - ‘vect_db.json’ saved [150192/150192]



In [ ]:
class CustomEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        return [modelo_embeddings.encode(text).tolist() for text in input]

In [ ]:
client = chromadb.PersistentClient(path="./bdd_vectorial")


collection = client.get_or_create_collection("document_chunks", embedding_function=CustomEmbeddingFunction())

with open('vect_db.json', 'r', encoding='utf-8') as file:
  data = json.load(file)

collection.add(
        ids=data["ids"],
        documents=data["documents"],
    )

In [ ]:
!wget https://raw.githubusercontent.com/nacho-gonz/NLP_TUIA_2024/refs/heads/main/bdd_grafos.rdf

--2024-12-17 21:37:31--  https://raw.githubusercontent.com/nacho-gonz/NLP_TUIA_2024/refs/heads/main/bdd_grafos.rdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6384 (6.2K) [text/plain]
Saving to: ‘bdd_grafos.rdf’

bdd_grafos.rdf      100%[===================>]   6.23K  --.-KB/s    in 0s      

2024-12-17 21:37:31 (97.9 MB/s) - ‘bdd_grafos.rdf’ saved [6384/6384]



In [ ]:
g = Graph()

file_path = "bdd_grafos.rdf"
g.parse(file_path, format="turtle")

<Graph identifier=N2f026081d1c64a83b05e9c08e1c0c2de (<class 'rdflib.graph.Graph'>)>

In [ ]:
!wget https://raw.githubusercontent.com/nacho-gonz/NLP_TUIA_2024/refs/heads/main/bdd_tabular.csv

--2024-12-17 21:37:32--  https://raw.githubusercontent.com/nacho-gonz/NLP_TUIA_2024/refs/heads/main/bdd_tabular.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 194 [text/plain]
Saving to: ‘bdd_tabular.csv’

bdd_tabular.csv     100%[===================>]     194  --.-KB/s    in 0s      

2024-12-17 21:37:32 (15.4 MB/s) - ‘bdd_tabular.csv’ saved [194/194]



In [ ]:
bdd_tabular = pd.read_csv('bdd_tabular.csv')
bdd_tabular.rename(columns={'All Time Plays':'Total Players','This Month':'Month Players','Weight':'Difficulty'}, inplace=True)

### Funciones de búsqueda y limpieza

In [ ]:
def limpieza_texto_pc(text:str):
  text = text.lower()

  # Eliminar acentos
  text = unidecode(text)

  # Eliminar caracteres especiales pero mantener espacios
  text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)

  # Eliminar espacios múltiples
  text = re.sub(r'\s+', ' ', text).strip()

  return text

#### Búsqueda en base de datos de grafos

In [ ]:
def buscar_grafos(query:str, model:str):
  messages = [
    {
      "role":"system",
      "content":f""" You are an assistant that will be responding only RDF querys.
      """
    },
    {
      "role": "user",
      "content": f""" I have a RDF graph database with the following format: {g.serialize(format="turtle")}. Always follow the database format.
    I want you to write a query in RDF to answer the following question:
    '{query}'.

    Return me only the query in RDF, without explanation or additional text.
    Make sure you only return clean, functional code, without characters like \\n or \``` at the beginning or end. Be sure to not add characters or words before or after the query.
    """
    }
  ]

  completion = llm_client.chat.completions.create(
      model=model,
    messages=messages,
    max_tokens=500
  )
  results = g.query(completion.choices[0].message.content)
  text_context = []
  for row in results:
    for key, value in row.asdict().items():  # Convierte el ResultRow a un diccionario
      text_context.append(str(value).replace("http://rajas_data.org", ""))
  contexto = ', '.join(text_context)
  contexto = contexto.replace('_', ' ')
  return unquote(contexto)


#### Búsqueda en base de datos tabular

In [ ]:
def buscar_tabular(query:str,model:str):
  messages = [
    {
      "role": "user",
      "content": f""" I have a Pandas DataFrame with the following columns: {list(bdd_tabular.columns)}. The DataFrame counts with only one row, so don't add more commands to the query than the column name.
    I want you to write a query in pandas to answer the following question:
    '{query}'.

    Return me only the query in pandas, without explanation or additional text. The database name is: bdd_tabular. Be sure to use this name in the query.
    Make sure you only return clean, functional code, without characters like \\n at the beginning or end. Be sure to not add characters or words before or after the query.
    Don't change the columns names, their uppercases characters or anything.
    """
    }
  ]

  completion = llm_client.chat.completions.create(
      model=model,
    messages=messages,
    max_tokens=500
  )

  consulta = completion.choices[0].message.content
  try:
    contexto = eval(consulta).iloc[0]
  except:
    contexto = eval(consulta)

  return contexto


#### Búsqueda en base de datos vectorial

In [ ]:
chunks_pc = collection.get()['documents']
chunks_limpios = [limpieza_texto_pc(chunk) for chunk in chunks_pc]
chunks_tokenizados = [word_tokenize(chunk) for chunk in chunks_limpios]
bm25 = BM25Okapi(chunks_tokenizados)

In [ ]:
def buscar_con_pc(query:str):
  processed_query = limpieza_texto_pc(query)
  tokenized_query = word_tokenize(processed_query)
  scores = bm25.get_scores(tokenized_query)
  top_indices = np.argsort(scores)[::-1][:5]
  top_chunks = [chunks_pc[i] for i in top_indices]

  return top_chunks

In [ ]:
def buscar_semantica(query:str):
  embeddings = modelo_embeddings.encode([query])
  results = collection.query(
      query_embeddings=embeddings,
      n_results=5
  )
  return results['documents']

In [ ]:
def buscar_texto(query:str):
  busqueda_pc = buscar_con_pc(query=query)
  busqueda_semantica = buscar_semantica(query=query)

  consultas_pasajes = []

  for pasaje_pc in busqueda_pc:
    consultas_pasajes.append([query, pasaje_pc])

  for pasaje_semantica in busqueda_semantica[0]:
    consultas_pasajes.append([query, pasaje_semantica])

  puntuaciones = reranker.compute_score(consultas_pasajes)

  mejor_score = max(puntuaciones)
  indice_mejor_score = puntuaciones.index(mejor_score)
  mejor_consulta = consultas_pasajes[indice_mejor_score]
  return mejor_consulta


## Charla con Chatbot

In [ ]:
def buscar_info(clase:str, query:str, model:str):
  if clase == 'relations':
    resultado = buscar_grafos(query=query, model=model)
  elif clase == 'statistics':
    resultado = buscar_tabular(query=query, model=model)
  else:
    resultado = buscar_texto(query=query)

  return resultado

In [ ]:
def hablar_chatbot(model:str):
  query = input('Ingrese su pregunta de Rajas of the Ganges: ')

  idioma = detect(query)
  if idioma == 'en':
    lang = 'english'
  elif idioma == 'es':
    query = GoogleTranslator(source='es', target='en').translate(query)
    lang = 'spanish'
  else:
    raise ValueError("Idioma no soportado. Solo inglés y español.")

  tipo_query = classify_query(query=query, model=model)
  context_data = buscar_info(clase=tipo_query, query=query, model=model)

  if tipo_query == 'relations' or tipo_query == 'statistics':
    context = context_data
  else:
    context = context_data[1]
  if not context:
    context_data = buscar_info(clase='abc', query=query, model=model)
    context = context_data[1]


  messages = [
    {
      "role": "system",
      "content":f"""
      You are an assistant that will be expert in the boardgame Rajas of the Ganges, any question about an game would be about the Rajas.
      """
    },
    {
      "role": "user",
      "content": f""" Answer the question. The answer needs to be in {lang}, clear and without extra information. with that context: {context}

      Question: {query}
      Answer:

      """
    }
  ]

  completion = llm_client.chat.completions.create(
      model=model,
    messages=messages,
    max_tokens=500
  )

  return completion.choices[0].message.content

In [65]:
print(hablar_chatbot("Qwen/Qwen2.5-72B-Instruct"))

Ingrese su pregunta de Rajas of the Ganges: Como influencia el karma en el juego?
El karma te permite influir en el resultado de los dados. Puedes gastar 1 punto de karma para cambiar el número de cualquier dado en tu estatua de Kali al número opuesto (por ejemplo, un “1” se convierte en un “6” y viceversa). Comienzas con 1 punto de karma, pero puedes aumentarlo hasta un máximo de 3 puntos a lo largo del juego. Cada vez que usas un punto de karma, tu cubo de karma baja 1 nivel.


# Ejercicio 2 - Agente

In [ ]:
# Descarga de Ollama
!curl -fsSL https://ollama.com/install.sh | sh

# Iniciamos Ollama en background
!rm -f ollama_start.sh
!echo '#!/bin/bash' > ollama_start.sh
!echo 'ollama serve' >> ollama_start.sh
# Make the script executable
!chmod +x ollama_start.sh
!nohup ./ollama_start.sh &

!ollama pull llama3.2 > ollama.log

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
nohup: appending output to 'nohup.out'
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulli

In [ ]:
# Configuración del logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [ ]:
# def search_wikipedia(query: str, lang: str = 'es') -> str:
#     query = f"{query} para el juego Rajas of the Ganjes"
#     try:
#         wikipedia.set_lang(lang)
#         result = wikipedia.summary(query, sentences=3)
#         return result
#     except Exception as e:
#         return "No se encontró información"


def search_in_graph(query):
  context = buscar_grafos(query, "Qwen/Qwen2.5-72B-Instruct")
  return context


def search_in_tabular(query):
  context = buscar_tabular(query, "Qwen/Qwen2.5-72B-Instruct")
  return context


def search_in_vectorial(query):
  context = buscar_texto(query)
  return context


In [ ]:
tools_list = [
    FunctionTool.from_defaults(fn=search_in_graph, description="Search for information in the graph database. Use: search text"),
    FunctionTool.from_defaults(fn=search_in_tabular, description="Search for information in the tabular database. Usar: search text"),
    FunctionTool.from_defaults(fn=search_in_vectorial, description="Search for information in the vectorial/text database. Usar: search text"),
    # FunctionTool.from_defaults(fn=search_wikipedia, description="Search for information on Wikipedia. Usar: search text"),
]

In [ ]:
modelo_llm_agente = Ollama(
    model="llama3.2:latest",
    request_timeout=500.0,
    temperature=0.1,
    context_window=4096
)

Settings.llm = modelo_llm_agente
!nohup litellm --model ollama/llama3.2:latest --port 8000 > litellm.log 2>&1 &

In [ ]:
react_agent = ReActAgent.from_tools(
    tools_list,
    llm=modelo_llm_agente,
    verbose=True,
    chat_formatter=ReActChatFormatter(),
    system_prompt="""You are an assistant that can respond in spanish or english depending on the query. You need to follow restrictively the next rules:

## Tools
You have access to a wide variety of tools. You are responsible for using
the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools
to complete each subtask.


Available tools:
{search_in_graph}: Action Input: "search text"
{search_in_tabular}: Action Input: "search text"
{search_in_vectorial}: Action Input: "search text"

## Tools themes:
- **search_in_graph**: DESIGNERS, ARTISTS, GAME LANGUAGES, PUBLISHERS, OTHER LANGUAGES, AWARDS, COUNTRY.
- **search_in_tabular**: STATISTICS IN GENERAL, RELEASE YEAR, MINIMUM AGE RECOMMENDED, TIME PLAY, AMOUNT OF PLAYERS.
- **search_in_vectorial**: GENERAL RULES, AUTOMATAS RULES, NAVARATNA RULES, GAME OBJECTIVE, COMPETITOR STRATEGY, BEGINNER STRATEGY, PORTUGUESE STRATEGY

## Tools Parameters:
- Always use only the query text to call the tools.
- Never use another information to be part of the parameters.
- The tools only accept strings in the parameters.

## Output Format
To answer the question, please use the following format.

```
Thought: I need to use a tool to help me answer the question
Action: tool_name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in literal string format.
```

Please ALWAYS start with a Thought.

Please use a valid string format for the Action Input. Do NOT do this {{'input': 'hello world', 'num_beams': 5}}.

If this format is used, the user will respond in the following format:

```
Observation: tool response
```

## Answer rules:
You should keep repeating the above format until you have enough information
to answer the question without using any more tools. At that point, you MUST respond
in the one of the following two formats:

```
Thought: I can answer without using any more tools.
Answer: [your answer here]
```

```
Thought: I cannot answer the question with the provided tools.
Answer: Sorry, I cannot answer your query.
```

## Aditional rules:
- Never modify the user query: Everytime you receive a new query, forget the previous one. The new query must be treated independently and without previous influences.
- Priorize the tools by the relation theme of the query. Secure yourself by selecting the best tool that adjust to the query with the proportioned themes.
- Never response with information that does not provide directly from the tools.
- The output format must be strict and follow the example proportioned.
- You MUST obey the function signature of each tool. Do NOT pass in no arguments if the function expects arguments.

## IMPORTANT:
- Always follow the exact format of the examples for each tool.
- If the query requires information of multiples tools, you MUST use ALL the tolls needed to obtain a FULL answer
- Don't skip any relevant tool although you have part of the answer.
- Don't use the output of an tool in the input of another tool. Only use the user query to use the tools.


## Current Conversation
Below is the current conversation consisting of interleaving human and assistant messages.

"""
      ,
            react_chat_history=False,
          context="""
          You are an useful assistant that responds in spanish or english depending on the query. Remember the outputs of all the tools to use them in the final answer. You are an expert in the boardgame Rajas of the Ganges, all the questions about a game would be about the Rajas of the Ganges.
          """)


def chat_con_agente(query: str):
    """
    Función para interactuar con el agente ReAct.
    """
    try:
        if not query.strip():
            return "La consulta está vacía"

        response = react_agent.chat(query)
        logger.info(f"Consulta: {query}")
        logger.info(f"Respuesta: {response}")
        return response
    except Exception as e:
        return f"Error al procesar la consulta: {str(e)}"

def ejecutar_ejemplo():
    print("=== Ejemplo de interacción con el agente ReAct ===")

    query = "when the game was released?"
    response = chat_con_agente(query)
    print(f"Respuesta: {response}")


In [ ]:
if __name__ == "__main__":
    # Configurar el entorno de ejecución
    import logging
    logging.basicConfig(level=logging.INFO)

    # Ejecutar el ejemplo
    ejecutar_ejemplo()

=== Ejemplo de interacción con el agente ReAct ===
> Running step 8f065101-9046-4c7b-8674-9d18002b5712. Step input: when the game was released?
Thought: The current language of the user is: english. I need to use a tool to help me answer the question.
Action: search_in_graph
Action Input: {'properties': AttributedDict([('query', AttributedDict([('title', 'Rajas of the Ganges release date')]))]), 'required': ['query'], 'type': 'object'}
Observation: Error: search_in_graph() got an unexpected keyword argument 'properties'
> Running step d9a8379f-2252-4fda-ab53-df2480df2b6b. Step input: None
Thought: The current language of the user is: english. I need to use a tool to help me answer the question.
Action: search_in_graph
Action Input: {'query': 'Rajas of the Ganges release date'}
Observation: 
> Running step 09ad7a36-9fa2-44eb-9612-e9779ef5a667. Step input: None
Thought: The current language of the user is: english. It seems that the search_in_graph tool did not provide a useful response.